In [ ]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing import cpu_count

from sklearn.model_selection import TimeSeriesSplit, train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline

import xgboost as xgb  # for final medal predictions
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from statsmodels.tsa.holtwinters import Holt
import scipy

# ------------------------------------------------------------------
# 1) Enable multi-threading in TensorFlow
# ------------------------------------------------------------------
os.environ["OMP_NUM_THREADS"] = str(cpu_count())
os.environ["MKL_NUM_THREADS"] = str(cpu_count())
tf.config.threading.set_intra_op_parallelism_threads(cpu_count())
tf.config.threading.set_inter_op_parallelism_threads(cpu_count())

# ------------------------------------------------------------------
# 2) Load & Basic Cleaning
# ------------------------------------------------------------------
data_path = "MCM_2025\\2025_MCM-ICM_Problems\\2025_Problem_C_Data\\summerOly_athletes.csv"
df = pd.read_csv(data_path)

# Remove invalid Team entries
df = df[~df['Team'].str.contains(r'\d|\.', na=False)]
roman_pattern = r'(?:\s|-)M{0,4}(?:CM|CD|D?C{0,3})(?:XC|XL|L?X{0,3})(?:IX|IV|V?I{0,3})(?:\s|$)'
df = df[~df['Team'].str.contains(roman_pattern, na=False, flags=re.IGNORECASE)]

# Remove countries not present in last two Olympics (2020 & 2024)
recent_years = [2020, 2024]
df = df[df["NOC"].isin(df[df["Year"].isin(recent_years)]["NOC"].unique())]

# Remove events that appeared <4 times
event_counts = df["Event"].value_counts()
df = df[df["Event"].isin(event_counts[event_counts >= 4].index)]

# Convert Year to int
df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype(int)

# ------------------------------------------------------------------
# 3) Feature Engineering (Row-level)
# ------------------------------------------------------------------
df["Seed_Player"] = df.groupby("Name")["Medal"].transform(
    lambda x: (x == "Gold").cumsum() + (x == "Silver").cumsum()
).fillna(0).astype(int)

df["Elite_Player"] = df.groupby("Name")["Medal"].transform(
    lambda x: max(0, (x != "No medal").sum() - 1 + (x == "Gold").sum())
).fillna(0).astype(int)

df["Participation_Count"] = df.groupby("Name")["Year"].transform("count")
df["Retiring_Player"] = df["Participation_Count"].apply(lambda x: max(0, x - 3)).astype(int)

host_cities = {2008: "CHN", 2012: "GBR", 2016: "BRA", 2020: "JPN", 2024: "FRA"}
df["Is_Host"] = df.apply(lambda row: 1 if host_cities.get(row['Year'], None) == row['NOC'] else 0, axis=1)

df["NOC"] = df["NOC"].astype(str)
df["Name"] = df["Name"].astype(str)

# ------------------------------------------------------------------
# 4) Aggregation by (Year, NOC)
# ------------------------------------------------------------------
df_num_cols = df.select_dtypes(include=["number"]).columns.difference(["Year"]).tolist()

df_noc_year = (
    df.groupby(["Year", "NOC"], as_index=False)
      .apply(lambda subdf: subdf[df_num_cols].sum(numeric_only=True))
      .reset_index(drop=True)
)

# Summation of medal counts
medal_counts = df.groupby(["Year", "NOC"])["Medal"].value_counts().unstack(fill_value=0).reset_index()
medal_counts.rename(columns={"Gold":"Medal_Gold","Silver":"Medal_Silver","Bronze":"Medal_Bronze"}, inplace=True)

df_noc_year = df_noc_year.merge(medal_counts, on=["Year","NOC"], how="left").fillna(0)

# ------------------------------------------------------------------
# 5) One-Hot Encoding NOC
# ------------------------------------------------------------------
ohe_noc = OneHotEncoder(handle_unknown="ignore", sparse=False)
encoded_noc = ohe_noc.fit_transform(df_noc_year[["NOC"]])
noc_ohe_cols = [f"NOC_{cat}" for cat in ohe_noc.categories_[0]]

df_noc_year.reset_index(drop=True,inplace=True)
noc_ohe_df = pd.DataFrame(encoded_noc, columns=noc_ohe_cols)
df_noc_year_ohe = pd.concat([df_noc_year.drop(columns=["NOC"]), noc_ohe_df], axis=1)

with open("noc_encoder.pkl","wb") as f:
    pickle.dump(ohe_noc,f)

# ------------------------------------------------------------------
# 6) Scaling
# ------------------------------------------------------------------
target_cols = ["Medal_Gold","Medal_Silver","Medal_Bronze"]
feature_cols = df_noc_year_ohe.columns.difference(["Year"]+target_cols)

scaler_features = StandardScaler()
scaler_targets = StandardScaler()

X_all = scaler_features.fit_transform(df_noc_year_ohe[feature_cols])
y_all = scaler_targets.fit_transform(df_noc_year_ohe[target_cols])

with open("scaler_features.pkl","wb") as f:
    pickle.dump(scaler_features,f)
with open("scaler_targets.pkl","wb") as f:
    pickle.dump(scaler_targets,f)

df_unscaled = df_noc_year_ohe.copy()  # aggregator unscaled

# ------------------------------------------------------------------
# Weighted sample approach
# We'll define sample_weight = 1 + (Gold + Silver + Bronze) from the unscaled aggregator
# so countries with bigger total medals get more weight
# We'll store them in an array that lines up with each row of X_all
# ------------------------------------------------------------------
row_sums = (df_noc_year["Medal_Gold"] + df_noc_year["Medal_Silver"] + df_noc_year["Medal_Bronze"]).values
sample_weight_all = 1.0 + row_sums  # shape = (num_rows,)

# ------------------------------------------------------------------
# LSTM Baseline code (unchanged, but with debug prints)
# ------------------------------------------------------------------
def build_lstm_baseline_model(x_vals, y_vals, n_lag=3):
    print(f"[LSTM Debug] Building baseline with LSTM: x_vals length={len(x_vals)}, n_lag={n_lag}")
    if len(x_vals) < n_lag+1:
        print("[LSTM Debug] Not enough data points for LSTM baseline.")
        return None

    x_vals = np.array(x_vals).reshape(-1,1)
    y_vals = np.array(y_vals).reshape(-1,1)

    print(f"[LSTM Debug] LSTM training data shape: x_vals={x_vals.shape}, y_vals={y_vals.shape}")

    # Scale y
    scaler = MinMaxScaler(feature_range=(0,1))
    y_scaled = scaler.fit_transform(y_vals)

    X, Y = [], []
    for i in range(n_lag, len(y_scaled)):
        X.append(y_scaled[i-n_lag:i])
        Y.append(y_scaled[i])
    X, Y = np.array(X), np.array(Y)
    print(f"[LSTM Debug] LSTM training sequences shape: X={X.shape}, Y={Y.shape}")

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_lag,1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    print(f"[LSTM Debug] Starting LSTM fit for {len(X)} samples.")
    model.fit(X, Y, epochs=50, batch_size=8, verbose=1)
    print("[LSTM Debug] LSTM training done.")

    return model, scaler

def predict_lstm_value(model, scaler, x_vals, y_vals, x_target, n_lag=3):
    y_vals = np.array(y_vals).reshape(-1,1)
    y_scaled = scaler.transform(y_vals)

    if len(y_scaled)<n_lag:
        return 0.0
    X_input = y_scaled[-n_lag:].reshape(1,n_lag,1)
    y_pred_sc= model.predict(X_input, verbose=0)
    y_pred= scaler.inverse_transform(y_pred_sc)
    return y_pred[0][0]

def build_baseline_model(x_vals, y_vals):
    """
    We'll only do LSTM baseline now, ignoring other approaches.
    """
    ret = build_lstm_baseline_model(x_vals, y_vals)
    if ret is None:
        return None
    return ("LSTM", ret)

def predict_baseline_value(model_info, x_target, x_vals, y_vals):
    if model_info is None:
        return 0.0
    name, (model, scaler) = model_info
    if name=="LSTM":
        return predict_lstm_value(model, scaler, x_vals, y_vals, x_target)
    return 0.0

# ------------------------------------------------------------------
# XGBoost Regressor for final predictions
# We'll train 3 separate models: one for Gold, Silver, Bronze
# ------------------------------------------------------------------
def train_xgboost_timeseries(X_train, Y_train, sample_weight, n_splits=3):
    """
    X_train: shape (N, num_features)
    Y_train: shape (N,) for one target
    sample_weight: array of shape (N,) with the weighting
    We'll do a param grid search on e.g. max_depth, learning_rate, etc.
    We'll do TSCV to pick best hyperparams.
    """

    tscv = TimeSeriesSplit(n_splits=n_splits)

    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=-1, verbosity=0)
    param_grid = {
        'max_depth':[3,5],
        'learning_rate':[0.01,0.05,0.1],
        'n_estimators':[100,200],
        # add more if you want
    }

    # We'll do a manual approach to TSCV grid search since sklearn's GridSearchCV doesn't do advanced TS splits by default
    best_params= None
    best_score= float("inf")

    for md in param_grid['max_depth']:
        for lr in param_grid['learning_rate']:
            for nest in param_grid['n_estimators']:
                fold_mae=[]
                for fold,(tr_idx,val_idx) in enumerate(tscv.split(X_train)):
                    X_tr, X_val = X_train[tr_idx], X_train[val_idx]
                    y_tr, y_val = Y_train[tr_idx], Y_train[val_idx]
                    w_tr, w_val = sample_weight[tr_idx], sample_weight[val_idx]

                    model_ = xgb.XGBRegressor(objective='reg:squarederror',n_jobs=-1,
                                              max_depth=md, learning_rate=lr, n_estimators=nest)
                    model_.fit(X_tr, y_tr, sample_weight=w_tr, eval_set=[(X_val, y_val)],
                               eval_metric='mae', verbose=False)

                    val_preds= model_.predict(X_val)
                    mae_ = mean_absolute_error(y_val, val_preds, sample_weight=w_val)
                    fold_mae.append(mae_)
                mean_mae= np.mean(fold_mae)
                if mean_mae < best_score:
                    best_score= mean_mae
                    best_params= (md, lr, nest)
    print(f"[XGBoost TSCV] Best params => max_depth={best_params[0]}, lr={best_params[1]}, n_estimators={best_params[2]}, score={best_score:.3f}")

    # retrain final
    final_xgb= xgb.XGBRegressor(objective='reg:squarederror', n_jobs=-1,
                                max_depth=best_params[0], learning_rate=best_params[1], n_estimators=best_params[2])
    final_xgb.fit(X_train, Y_train, sample_weight=sample_weight, verbose=False)
    return final_xgb

def train_xgboost_up_to_year(X_all, y_all, df_agg, sample_weight_all, end_year=2024):
    """
    We'll train 3 separate models for each target: gold,silver,bronze.
    We'll do TSCV with the portion of data where Year<end_year
    """
    mask= (df_agg["Year"]<end_year)
    X_sub= X_all[mask]
    sw_sub= sample_weight_all[mask]

    # We'll separate y_all columns => 3 separate arrays
    y_gold= y_all[mask,0]
    y_silver= y_all[mask,1]
    y_bronze= y_all[mask,2]

    # Train each with TSCV
    gold_model= train_xgboost_timeseries(X_sub, y_gold, sw_sub, n_splits=3)
    silver_model= train_xgboost_timeseries(X_sub, y_silver, sw_sub, n_splits=3)
    bronze_model= train_xgboost_timeseries(X_sub, y_bronze, sw_sub, n_splits=3)

    return gold_model, silver_model, bronze_model

def xgboost_predict_3models(models, X_input):
    """
    models = (gold_model, silver_model, bronze_model)
    """
    gold_model, silver_model, bronze_model = models
    pred_gold= gold_model.predict(X_input)
    pred_silver= silver_model.predict(X_input)
    pred_bronze= bronze_model.predict(X_input)
    preds= np.vstack([pred_gold,pred_silver,pred_bronze]).T
    return preds

# ------------------------------------------------------------------
# Workflow A for 2024
# ------------------------------------------------------------------
def build_baseline_and_predict_2024_xgboost(models_3, df_agg, feature_cols, target_cols, scaler_features, scaler_targets):
    """
    We build LSTM baseline for 2024, fill aggregator, then do xgboost_predict_3models.
    """
    df_2024= df_agg[df_agg["Year"]==2020].copy()
    df_2024["Year"]=2024

    noc_cols= [c for c in df_agg.columns if c.startswith("NOC_")]
    df_agg["NOC"] = df_agg[noc_cols].idxmax(axis=1).str.replace("NOC_","")
    df_2024["NOC"]= df_2024[noc_cols].idxmax(axis=1).str.replace("NOC_","")

    if "NOC_FRA" in df_2024.columns:
        df_2024["Is_Host"]= df_2024["NOC_FRA"]

    numeric_feats= [c for c in feature_cols if c not in noc_cols and c!="Year" and c!="Is_Host"]
    hist_data= df_agg[df_agg["Year"]<2024].copy()

    hist_data["NOC"]= hist_data[noc_cols].idxmax(axis=1).str.replace("NOC_","")

    for feat in numeric_feats:
        pivot_df= hist_data.pivot(index="NOC", columns="Year", values=feat).fillna(0)
        for idx_noc in pivot_df.index:
            xvals= np.array(pivot_df.columns, dtype=float)
            yvals= pivot_df.loc[idx_noc].values
            if len(xvals)<2:
                continue
            baseline_info= build_baseline_model(xvals,yvals)
            val_2024= predict_baseline_value(baseline_info,2024, xvals,yvals)
            noc_col= f"NOC_{idx_noc}"
            if noc_col in df_2024.columns:
                df_2024.loc[df_2024[noc_col]==1, feat] = val_2024

    # ensure columns
    for col in df_agg.columns:
        if col not in df_2024.columns:
            df_2024[col]=0

    X_2024_unsc= df_2024[feature_cols].values
    X_2024_sc= scaler_features.transform(X_2024_unsc)
    preds_2024_sc= xgboost_predict_3models(models_3, X_2024_sc)
    preds_2024= scaler_targets.inverse_transform(preds_2024_sc)

    # get real aggregator for 2024
    df_real_2024= df_agg[df_agg["Year"]==2024].reset_index(drop=True)
    df_real_2024["NOC"]= df_real_2024[noc_cols].idxmax(axis=1).str.replace("NOC_","")

    actual_2024= scaler_targets.inverse_transform(df_real_2024[target_cols].values)

    df_res_2024= pd.DataFrame({
        "NOC": df_real_2024["NOC"].astype(str),
        "Gold_Actual": actual_2024[:,0],
        "Silver_Actual": actual_2024[:,1],
        "Bronze_Actual": actual_2024[:,2],
        "Gold_Predicted": preds_2024[:,0],
        "Silver_Predicted": preds_2024[:,1],
        "Bronze_Predicted": preds_2024[:,2],
    })
    return df_res_2024

def visualize_2024_results(df_res_2024):
    df= df_res_2024.copy()
    df["Gold_Error_%"] = np.where(df["Gold_Actual"]>0, abs(df["Gold_Predicted"]-df["Gold_Actual"])/df["Gold_Actual"]*100,0)
    df["Silver_Error_%"] = np.where(df["Silver_Actual"]>0, abs(df["Silver_Predicted"]-df["Silver_Actual"])/df["Silver_Actual"]*100,0)
    df["Bronze_Error_%"] = np.where(df["Bronze_Actual"]>0, abs(df["Bronze_Predicted"]-df["Bronze_Actual"])/df["Bronze_Actual"]*100,0)

    error_cols= ["Gold_Error_%","Silver_Error_%","Bronze_Error_%"]
    filtered_df= df[(df["Gold_Actual"]>0)|(df["Silver_Actual"]>0)|(df["Bronze_Actual"]>0)]

    plt.figure(figsize=(18,6))
    for i,col in enumerate(error_cols):
        plt.subplot(1,3,i+1)
        plt.hist(filtered_df[col], bins=50, range=(0,100), edgecolor='black', alpha=0.7)
        plt.title(f"Distribution of {col}")
        plt.xlabel("Error %")
        plt.ylabel("Frequency")
        plt.grid(True)
    plt.tight_layout()
    plt.show()

    exceed_100= (filtered_df[error_cols]>100).sum()/len(filtered_df)*100
    print("Pct of data >100% error:", exceed_100)

    filtered_df["Total_Actual"]=filtered_df["Gold_Actual"]+filtered_df["Silver_Actual"]+filtered_df["Bronze_Actual"]
    filtered_df.sort_values(by="Total_Actual", ascending=False, inplace=True)

    nocs=filtered_df["NOC"].values
    gold_act= filtered_df["Gold_Actual"].values
    gold_pred= filtered_df["Gold_Predicted"].values
    silver_act= filtered_df["Silver_Actual"].values
    silver_pred= filtered_df["Silver_Predicted"].values
    bronze_act= filtered_df["Bronze_Actual"].values
    bronze_pred= filtered_df["Bronze_Predicted"].values

    # gold line
    plt.figure(figsize=(12,4))
    plt.plot(nocs,gold_act,label="ActualGold",marker='o')
    plt.plot(nocs,gold_pred,label="PredGold",marker='x')
    plt.xticks(rotation=90)
    plt.xlabel("NOC")
    plt.ylabel("Gold Medals")
    plt.title("Gold: Actual vs Pred (2024, LSTM baseline + XGBoost)")
    plt.legend()
    plt.grid()
    plt.show()

    # silver line
    plt.figure(figsize=(12,4))
    plt.plot(nocs,silver_act,label="ActualSilver",marker='o')
    plt.plot(nocs,silver_pred,label="PredSilver",marker='x')
    plt.xticks(rotation=90)
    plt.xlabel("NOC")
    plt.ylabel("Silver Medals")
    plt.title("Silver: Actual vs Pred (2024, LSTM baseline + XGBoost)")
    plt.legend()
    plt.grid()
    plt.show()

    # bronze line
    plt.figure(figsize=(12,4))
    plt.plot(nocs,bronze_act,label="ActualBronze",marker='o')
    plt.plot(nocs,bronze_pred,label="PredBronze",marker='x')
    plt.xticks(rotation=90)
    plt.xlabel("NOC")
    plt.ylabel("Bronze Medals")
    plt.title("Bronze: Actual vs Pred (2024, LSTM baseline + XGBoost)")
    plt.legend()
    plt.grid()
    plt.show()

    # final MAE
    gold_mae= mean_absolute_error(gold_act,gold_pred)
    silver_mae= mean_absolute_error(silver_act,silver_pred)
    bronze_mae= mean_absolute_error(bronze_act,bronze_pred)
    print(f"Gold MAE: {gold_mae:.2f}")
    print(f"Silver MAE: {silver_mae:.2f}")
    print(f"Bronze MAE: {bronze_mae:.2f}")

# Workflow B for 2028
def build_2028_baseline_and_predict_xgboost(models_3, df_agg, feature_cols, target_cols, scaler_features, scaler_targets):
    df_2028= df_agg[df_agg["Year"]==2024].copy()
    df_2028["Year"]=2028

    if "NOC_USA" in df_2028.columns:
        df_2028["Is_Host"]= df_2028["NOC_USA"]

    noc_cols= [c for c in df_agg.columns if c.startswith("NOC_")]
    numeric_feats= [c for c in feature_cols if c not in noc_cols and c!="Year" and c!="Is_Host"]

    df_agg["NOC"]= df_agg[noc_cols].idxmax(axis=1).str.replace("NOC_","")

    for feat in numeric_feats:
        pivot_df= df_agg.pivot(index="NOC", columns="Year", values=feat).fillna(0)
        for idx_noc in pivot_df.index:
            xvals= np.array(pivot_df.columns,dtype=float)
            yvals= pivot_df.loc[idx_noc].values
            if len(xvals)<2:
                continue
            baseline_info= build_baseline_model(xvals,yvals)
            val_2028= predict_baseline_value(baseline_info,2028,xvals,yvals)
            noc_col= f"NOC_{idx_noc}"
            if noc_col in df_2028.columns:
                df_2028.loc[df_2028[noc_col]==1, feat]= val_2028

    for col in df_agg.columns:
        if col not in df_2028.columns:
            df_2028[col]=0

    X_2028_unsc= df_2028[feature_cols].values
    X_2028_sc= scaler_features.transform(X_2028_unsc)
    preds_2028_sc= xgboost_predict_3models(models_3, X_2028_sc)
    preds_2028= scaler_targets.inverse_transform(preds_2028_sc)

    df_2024_rows= df_noc_year[df_noc_year["Year"]==2024].reset_index(drop=True)

    df_res_2028= pd.DataFrame({
        "NOC": df_2024_rows["NOC"].values,
        "Gold_Predicted": preds_2028[:,0],
        "Silver_Predicted": preds_2028[:,1],
        "Bronze_Predicted": preds_2028[:,2],
    })
    print("\nPredicted medal counts for 2028 (XGBoost + LSTM baseline):")
    print(df_res_2028.head(20))
    return df_res_2028

# ------------------------------------------------------------------
# Final Execution
# ------------------------------------------------------------------

# 1) Train XGBoost up to <2024, do LSTM baseline for 2024
gold_2024, silver_2024, bronze_2024 = train_xgboost_up_to_year(X_all, y_all, df_unscaled, sample_weight_all, end_year=2024)
models_2024 = (gold_2024, silver_2024, bronze_2024)

df_2024_res= build_baseline_and_predict_2024_xgboost(models_2024, df_unscaled, feature_cols, target_cols,
                                                     scaler_features, scaler_targets)
print("\n**2024 Extrapolated Results (LSTM Baseline + XGBoost)**")
print(df_2024_res.head(30))
visualize_2024_results(df_2024_res)

# 2) Train XGBoost up to <2028, do LSTM baseline for 2028
gold_2028, silver_2028, bronze_2028 = train_xgboost_up_to_year(X_all, y_all, df_unscaled, sample_weight_all, end_year=2028)
models_2028= (gold_2028, silver_2028, bronze_2028)

df_2028_res= build_2028_baseline_and_predict_xgboost(models_2028, df_unscaled, feature_cols, target_cols,
                                                     scaler_features, scaler_targets)
print("\n--- Done with 2024 & 2028 XGBoost workflows. ---")